In [1]:
from petrify.space import Vector, Point, Plane
from petrify import tau, plane, render

## Complex Object Construction

Planar polygons are often the building block for forming complex solids. One of the most advanced versions of this is the generalized `Extrusion` class. Its input is an array of planar polygons with the same number of points on each "slice":

In [2]:
from petrify.solid import Extrusion, PlanarPolygon, Basis
square = plane.Polygon([
    plane.Point(0, 0),
    plane.Point(0, 1),
    plane.Point(1, 1),
    plane.Point(1, 0)
])

dz = Vector(0, 0, 1)
spun_squares = [
    PlanarPolygon(Basis.xy + dz * ix, square * plane.Matrix.rotate(tau / 16 * ix)) 
    for ix in range(8)
]

render(spun_squares)

Preview(child=LineSegments(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(64, 3), dty…

This class constructs rings to connect polygons with the same number of points, and generates endcaps to make the resultant solid "watertight":

In [3]:
render(Extrusion(spun_squares))

Preview(child=BufferGeometry(attributes={'position': <BufferAttribute shape=(180, 3), dtype=float32>, 'normal'…

Another powerful method for creating objects with rotational symmetry is the `Spun` solid. It takes an array of profiles with the same number of points, and "spins" them, equally rotating each in turn about an axis.

In [4]:
from petrify.solid import Spun

notch = plane.Polygon([
    plane.Point(0, 0),
    plane.Point(1, 0),
    plane.Point(1, 1),
    plane.Point(2, 2),
    plane.Point(1, 3),
    plane.Point(1, 4),
    plane.Point(0, 4)
])

axis = Vector.basis.z
start = Vector.basis.x
notched = Spun(axis, start, [notch] * 10)

render(notched.profiles())

Preview(child=LineSegments(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(140, 3), dt…

In [5]:
render(notched)

Preview(child=BufferGeometry(attributes={'position': <BufferAttribute shape=(270, 3), dtype=float32>, 'normal'…